# Separate data into different positions and do feature selection in order to train different models

In [1]:
from src.features.data_engineering import get_merged_seasons_data

In [2]:
data = get_merged_seasons_data()

## Separate data into different positions

In [5]:
data_gk = data[data['position'] == 'GK']
data_def = data[data['position'] == 'DEF']
data_mid = data[data['position'] == 'MID']
data_fd = data[data['position'] == 'FWD']

In [9]:
data_gk

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,season
2,Ron-Robert_Zieler,0,0,0,0,0.0,166,4,0,0,...,0,0,0,0,45,False,0,1,GK,2016-17
13,Allan_McGregor,0,0,0,0,0.0,146,4,0,0,...,0,0,0,0,45,True,0,1,GK,2016-17
23,Eldin_Jakupovic,0,0,16,0,0.0,147,4,1,0,...,3,0,0,0,40,True,0,1,GK,2016-17
26,Dusan_Kuciak,0,0,0,0,0.0,517,4,0,0,...,0,0,0,0,40,True,0,1,GK,2016-17
28,Kasper_Schmeichel,0,0,14,0,0.0,165,4,2,0,...,2,0,0,0,50,False,0,1,GK,2016-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133569,Joe Hart,0,0,0,0,0.0,352,298,0,0,...,0,-4,0,4,45,True,0,30,GK,2021-22
133571,Alphonse Areola,0,0,0,0,0.0,489,298,0,0,...,0,60,432,372,47,False,0,30,GK,2021-22
133581,Darren Randolph,0,0,0,0,0.0,407,298,0,0,...,0,114,203,89,43,False,0,30,GK,2021-22
133582,Pierluigi Gollini,0,0,0,0,0.0,486,298,0,0,...,0,-105,39,144,43,True,0,30,GK,2021-22
